In [1]:
import casadi
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML, display
from matplotlib.animation import FuncAnimation

In [ ]:
# System parameters
m = 1.0  # mass

# Define optimization variables
T = 5.0  # Time horizon
N = 100  # Number of control intervals
dt = T / N

# define dynamics
nq = 1  # position
nv = 1  # velocity
cq = casadi.SX.sym("q", nq)  # position
cv = casadi.SX.sym("v", nv)  # velocity
cu = casadi.SX.sym("u", nv)  # force input
aba_fn = casadi.Function("aba_fn", [cq, cv, cu], [cu / m])  # a = F/m


# Euler integrator
def euler_integrate(q, v, u):
    q_next = q + v * dt
    v_next = v + aba_fn(q, v, u) * dt
    return q_next, v_next


# Set up optimization
opti = casadi.Opti()

Q = opti.variable(nq, N + 1)
V = opti.variable(nv, N + 1)
U = opti.variable(nv, N)

# Set initial state
opti.subject_to(Q[:, 0] == np.array([0]))  # start at origin
opti.subject_to(V[:, 0] == np.array([0]))  # start at rest

# Set final state
opti.subject_to(Q[:, N] == np.array([1]))  # end at position 1
opti.subject_to(V[:, N] == np.array([0]))  # end at rest

# Set dynamics constraints
for k in range(N):
    q_next, v_next = euler_integrate(Q[:, k], V[:, k], U[:, k])
    opti.subject_to(Q[:, k + 1] == q_next)
    opti.subject_to(V[:, k + 1] == v_next)

# Set control limits
limit = 3.0
opti.subject_to(opti.bounded(-limit, U, limit))

# Define objective (minimize control effort and tracking error)
obj = 0
for k in range(N):
    obj += U[:, k].T @ U[:, k]  # control effort
    obj += (Q[:, k] - np.array([1])) ** 2  # tracking error

opti.minimize(obj)

# Solve
opti.solver("ipopt")
sol = opti.solve()

# Plot phase space
Q_traj = sol.value(Q)
V_traj = sol.value(V)

In [ ]:
# Animate
fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(111)
(line,) = ax.plot([], [], "bo-", linewidth=2, markersize=20)
ax.grid(True)
ax.set_xlim(-0.5, 1.5)
ax.set_ylim(-0.5, 0.5)


def update(frame):
    line.set_data([Q_traj[frame]], [0])
    return (line,)


anim = FuncAnimation(fig, update, frames=N + 1, interval=dt * 1000, blit=True)
plt.close()

html = HTML(anim.to_jshtml())
display(html)

In [ ]:
## Create a plot showing how the com has been moving over time and control input
fontsize = 15
save_fig = False
fig, ax1 = plt.subplots(figsize=(10,6))

# Plot position on left y-axis
ax1.plot([dt * i for i in range(len(Q_traj))], Q_traj, label='Position', linewidth=3, c="blue")
ax1.set_xlabel('Time', fontsize=fontsize)
ax1.set_ylabel('Position', fontsize=fontsize, color='blue')
ax1.tick_params(axis='y', labelcolor='blue', labelsize=fontsize)
ax1.tick_params(axis='x', labelsize=fontsize)

# Create second y-axis and plot control input
ax2 = ax1.twinx()
U_traj = sol.value(U)
ax2.plot([dt * i for i in range(len(U_traj))], U_traj.flatten(), label='Control Input', 
         linewidth=3, c="red", linestyle='--')
ax2.set_ylabel('Control Input', fontsize=fontsize, color='red')
ax2.tick_params(axis='y', labelcolor='red', labelsize=fontsize)

# Add grid and styling
ax1.grid(True, which='both', linestyle=':', linewidth=0.5)
ax1.minorticks_on()

# Title and legend
plt.title('Center of Mass Position and Control Input Over Time', fontsize=fontsize)
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, fontsize=fontsize, loc='upper right', bbox_to_anchor=(0.95, 0.6))

if save_fig:
    plt.savefig('com_position_and_control.png', bbox_inches='tight')
plt.show()
